# Final Assignment for Applied Data Science

### Intoduction

In our days there is a trend to travel overseas to enjoy new food, as a food 
enthusiastic I would like to know if a certain city has similar cuisines as in
my country. Usually this requires me to go and search the web for restaurants in the 
country that i'm traveling to.
I will try to tackle the problem using the Foursquare data to figure the similarity
in cuisines between two cities.



### Data

The data that i will get from the API will give me the restaurants in the area 
that i'm looking to travel.  
For each of the restaurants i will extract the followings:
1. Category
2. Location
3. Name 

I will target the base city as Toronto and will check how similar the Chicago city.

In [7]:
import requests
C_ID = ""
C_SEC = ""
V = "20180323"
LIMIT = 0
PREFIX = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}".format(C_ID, C_SEC, V)

query = "restaurants"
PREFIX = PREFIX + "&query={}&limit={}".format(query, LIMIT)
base_city = "Toronto, CA"
similar_to = ["Chicago, IL"]

In [8]:
def get_response(query):
    response = requests.get(query)
    response.raise_for_status()
    data = response.json()
    res = data['response']
    return res

Let's get the restaurants from the API

In [ ]:
place = base_city
new_query = PREFIX
new_query += "&near={}".format(place)
res = get_response(new_query)

iterating over the restaurants we got and extract the data we stated before

In [ ]:
# getting the latitude and longitude of the city
center_of_city = res['geocode']['center']
items = res['groups'][0]['items']

lats = []
lngs= []
names = []
categories = []

for item in items:
    venue = item['venue']
    lats.append(venue['location']['lat'])
    lngs.append(venue['location']['lng'])
    names.append(venue['name'])
    categories.append(venue['categories'][0]['shortName'])

df = pd.DataFrame({
    "Name": names,
    "Category": categories,
    "Latitude": lats,
    "Longitude": lngs
})
df.to_csv(place.split(",")[0] + ".csv")

Using this table i will see the percentage of each category in this area (the range will be 0-1).

## Methodology

In this project we will try to determine whether a certian city has similar cuisines to another.  
Since there are a lot of countries we will try to determine the similarity between Toronto, CA and Chicago, IL.

In the previous section we have collected the necessary data and saved into csv files.

The next section will focus on analyzing and adjusting the tables to include relevant indicators.

In the last section we will try to implement a machine learning algorithm in order to find the similarity
between two cities, i will take into account the frequncy of the cuisine in the area and will try to cluster 

In [31]:
df = pd.read_csv('Chicago.csv')[['Name', 'Category', 'Latitude', 'Longitude']]
values_cout = df['Category'].value_counts()
df_dict = pd.DataFrame(values_cout).to_dict()['Category']
sum_counts = sum(df_dict.values())
values = [x / sum_counts for x in df_dict.values()]
_df = pd.DataFrame({
    "Category": list(df_dict.keys()),
    "Count": values
})
_df

,Category,Count
0,Pizza,0.11
1,New American,0.10
2,Café,0.09
3,Restaurant,0.06
4,Burgers,0.05
5,Bakery,0.05
6,Deli / Bodega,0.05
7,French,0.04
8,Seafood,0.03
9,Donuts,0.03


After getting the data for each city i will intersect the tables and get a list of similar cuisines.
I will add the percentages in the list according to the table of the city that i'm comparing to.

The closer we get to 1.0 the similar the cities are.